# Day 7:  The Sum of Its Parts
[link](https://adventofcode.com/2018/day/7)

## Part 1: Order of steps

In [1]:
import re

def parse(lines):
  instructions = []
  for line in lines:
    match = re.search(r'Step ([A-Z]) must be finished before step ([A-Z]) can begin', line.strip())
    assert match, f'Wrong format in line "{line}"'
    before, after = match.group(1), match.group(2)
    instructions.append((before, after))
  return instructions

In [2]:
def get_all_steps(instructions):
  steps = set()
  for before, after in instructions:
    steps.add(before)
    steps.add(after)
  return sorted(list(steps))

def get_requirements(instructions):
  requirements = {s:set() for s in get_all_steps(instructions)}
  for required,step in instructions:
    requirements[step].add(required)
  return requirements

In [3]:
def get_available_steps(done, remaining, requirements):
  available = [s for s in remaining if not (s in requirements and requirements[s] - set(done))]
  assert available, f'Cannot find any suitable next steps'
  return available

def get_steps_order(instructions):
  requirements,remaining,done = get_requirements(instructions),get_all_steps(instructions),[]

  while remaining:
    all_available = get_available_steps(done, remaining, requirements)
    next_step = all_available[0]
    done.append(next_step)
    remaining.remove(next_step)

  return ''.join(done)

In [4]:
test_lines = [
  'Step C must be finished before step A can begin.',
  'Step C must be finished before step F can begin.',
  'Step A must be finished before step B can begin.',
  'Step A must be finished before step D can begin.',
  'Step B must be finished before step E can begin.',
  'Step D must be finished before step E can begin.',
  'Step F must be finished before step E can begin.'
]

test_instructions = parse(test_lines)
assert get_steps_order(test_instructions) == 'CABDFE'

In [5]:
with open('07 input.txt', 'r') as file:
  puzzle_input = parse(file)
len(puzzle_input)

101

In [6]:
solution_1 = get_steps_order(puzzle_input)
assert solution_1 == 'EPWCFXKISTZVJHDGNABLQYMORU'
solution_1

'EPWCFXKISTZVJHDGNABLQYMORU'

**Part 1 correct answer:** `EPWCFXKISTZVJHDGNABLQYMORU`

## Part 2: How long will it take to complete the steps?
With `5` workers and the `60`+ second step durations (step `A` takes `61` seconds, etc.)

[Critical Path Method](https://en.wikipedia.org/wiki/Critical_path_method) is used for choosing the steps that have the longest "critical path" duration.

**Update**: this was not needed as the task says: *If multiple steps are available, workers should still begin them in **alphabetical order**.*

Critical Path implementation:

```python
def get_paths(step, path=None, paths=None):
  if path is None:
    path = [r for r in remaining if r[0]==step]
  if paths is None:
    paths = []
  next_steps = [r for r in remaining if step in requirements[r[0]]]
  if len(next_steps)>0:
    for f in next_steps:
      new_path = path + [f]
      get_paths(f[0], new_path, paths)
  else:
    paths.append(path)
  return paths

# Critical path time = minimal time to get from the step to finish
def crit_time(step):
  return min(sum(s[1] for s in p) for p in get_paths(step))

print(' | '.join(f'{s}({crit_time(s)})' for s in get_all_steps(instructions)))
```

In [7]:
def complete_time(instructions, duration, workers_number, debug=False):
  remaining_steps = get_all_steps(instructions)
  remaining_step_time = {s:ord(s)-ord('A')+duration for s in remaining_steps}
  requirements = get_requirements(instructions)
  workers = ['.'] * workers_number
  time, done = 0, []

  if debug:
    print('\n S | 1 | 2 | Done      | Remaining')
    print(f'---|---|---|-----------|{"".rjust(30, "-")}')

  while remaining_steps:
    if '.' in workers: # There are workers doing nothing. Give them some steps to work on if there are any available.
      available_steps = [s for s in get_available_steps(done, remaining_steps, requirements) if s not in workers]
      free_woker_indexes = [i for i,s in enumerate(workers) if s is '.']
      available_steps = available_steps[:len(free_woker_indexes)]
      for i,available_step in enumerate(available_steps):
        workers[free_woker_indexes[i]] = available_step

    if debug:
      wks = " | ".join(workers)
      rmng = " ".join(f'{s}({remaining_step_time[s]})' for s in remaining_steps)
      print(f'{str(time).rjust(2)} | {wks} | {",".join(done).rjust(9)} | {rmng}')

    for worker_index,step in enumerate(workers):
      if step!='.':
        remaining_step_time[step] -= 1
        if remaining_step_time[step]==0:
          del remaining_step_time[step]
          workers[worker_index] = '.'
          done.append(step)
          remaining_steps.remove(step)

    time += 1

  return time

assert complete_time(test_instructions, 1, 2, True) == 15


 S | 1 | 2 | Done      | Remaining
---|---|---|-----------|------------------------------
 0 | C | . |           | A(1) B(2) C(3) D(4) E(5) F(6)
 1 | C | . |           | A(1) B(2) C(2) D(4) E(5) F(6)
 2 | C | . |           | A(1) B(2) C(1) D(4) E(5) F(6)
 3 | A | F |         C | A(1) B(2) D(4) E(5) F(6)
 4 | B | F |       C,A | B(2) D(4) E(5) F(5)
 5 | B | F |       C,A | B(1) D(4) E(5) F(4)
 6 | D | F |     C,A,B | D(4) E(5) F(3)
 7 | D | F |     C,A,B | D(3) E(5) F(2)
 8 | D | F |     C,A,B | D(2) E(5) F(1)
 9 | D | . |   C,A,B,F | D(1) E(5)
10 | E | . | C,A,B,F,D | E(5)
11 | E | . | C,A,B,F,D | E(4)
12 | E | . | C,A,B,F,D | E(3)
13 | E | . | C,A,B,F,D | E(2)
14 | E | . | C,A,B,F,D | E(1)


In [8]:
complete_time(puzzle_input, 61, 5)

952

**Incorrect answers:**
* `940` is too low
* `953` is too high

**Correct answer:** `952`